In [10]:
import keyboard
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import cv2
import tensorflow as tf
import vlc
import time


In [11]:
newmod=load_model("hand_gestures_15_DP_DA.h5")


In [19]:
cam = cv2.VideoCapture(0)

countr=0

class Control_Media:
    roi_top = 20
    roi_bottom = 300
    roi_right = 300
    roi_left = 600
    background = None
    accumulated_weight = 0.5
    vlc_instance = vlc.Instance()
    player = vlc_instance.media_player_new()
    media = vlc_instance.media_new("D:\Prateek\College\MDP\VID-20211019-WA0011.mp4")
    player.set_media(media)
    
    def __init__(self):
        pass
    
    def calc_accum_avg(self,frame, accumulated_weight):
        global background
        if Control_Media.background is None:
            background = frame.copy().astype("float")
            return None
        cv2.accumulateWeighted(frame, background, accumulated_weight)
    def segment(self,frame, threshold=20):
        global background
        diff = cv2.absdiff(background.astype("uint8"), frame)
        _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            Control_Media.player.play()
            return None
        else:
            hand_segment = max(contours, key=cv2.contourArea)
            return (thresholded, hand_segment)
    def thres_display(self,img):
        width=64
        height=64
        dim=(width,height)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        test_img=tf.keras.utils.img_to_array(resized)
        test_img=np.expand_dims(test_img,axis=0)
        result= newmod.predict(test_img)
        val=[index for index,value in enumerate(result[0]) if value ==1]
        return val
    def start_module(self):
        num_frames = 0
        while True:
            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break
            if keyboard.is_pressed('q'):
                break
            # get the current frame
            ret, frame = cam.read()
            frame = cv2.flip(frame, 1)
            frame_copy = frame

            # Grab the ROI from the frame
            roi = frame[Control_Media.roi_top:Control_Media.roi_bottom, Control_Media.roi_right:Control_Media.roi_left]

            # Apply grayscale and blur to ROI
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (7, 7), 0)
            
            cv2.rectangle(frame_copy, (Control_Media.roi_left, Control_Media.roi_top), (Control_Media.roi_right, Control_Media.roi_bottom), (0,0,255), 2)
            
            # Display the frame with segmented hand
            cv2.imshow("Hand Gestures", frame_copy)
            # For the first 30 frames we will calculate the average of the background.
            if num_frames < 60:
                self.calc_accum_avg(gray, Control_Media.accumulated_weight)
                if num_frames <= 59:
                    cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            else:
                cv2.putText(frame_copy, "Place your hand in side the box", (330, 340), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 0: Fist", (330, 355), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 1: Five", (330, 370), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 2: None", (330, 385), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 3: Okay", (330, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 4: Peace", (330, 415), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 5: Rad", (330, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 6: Straight", (330, 445), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                cv2.putText(frame_copy, "Index 7: Thumbs", (330, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1)
                
                hand = self.segment(gray)
                
                if hand is not None:
                    thresholded, hand_segment = hand
                    # Draw contours around hand segment
                    cv2.drawContours(frame_copy, [hand_segment + (Control_Media.roi_right, Control_Media.roi_top)], -1, (255, 0, 0),1)

                    # display the thresholded image
                    cv2.imshow("Thresholded Image", thresholded)
                    res=self.thres_display(thresholded)

                    if len(res)==0:
                        cv2.putText(frame_copy, str('None'), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                    else:
                        x='index'+str(res[0])
                        cv2.putText(frame_copy, str(x), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                        if str(res[0]) == '5':
                            Control_Media.player.set_pause(1)
                            print("pause")
                        elif str(res[0]) == '1':
                            current_time = Control_Media.player.get_time()
                            Control_Media.player.set_time(current_time + 250)
                            print("forward")
                        elif str(res[0]) == '7': 
                            current_time = Control_Media.player.get_time()
                            Control_Media.player.set_time(current_time - 250)
                            print("backward")
                        elif str(res[0]) == '4':
                            value = Control_Media.player.audio_get_volume()
                            print(value)
                            Control_Media.player.audio_set_volume(value - 2)
                            print("Vol_down")
                        elif str(res[0]) == '3':
                            value = Control_Media.player.audio_get_volume()
                            print(value)
                            Control_Media.player.audio_set_volume(value + 2)
                            print("Vol_up")
                        elif str(res[0]) == '2':
                            Control_Media.player.play()
#                         elif str(res[0]) == '5':
#                             Control_Media.player.stop()
                
            # increment the number of frames for tracking
            num_frames += 1  
                # Display the frame with segmented hand
            cv2.imshow("Hand Gestures", frame_copy)

            # Close windows with Esc

        Control_Media.player.stop()
        # Release the camera and destroy all the windows
        cam.release()
        cv2.destroyAllWindows()

In [20]:
obj=Control_Media()
obj.start_module()

1/1 [==============================] - 0s 19ms/step
200
Vol_down
1/1 [==============================] - 0s 21ms/step
198
Vol_down
1/1 [==============================] - 0s 20ms/step
196
Vol_down
1/1 [==============================] - 0s 20ms/step
194
Vol_down
1/1 [==============================] - 0s 23ms/step
192
Vol_down
1/1 [==============================] - 0s 21ms/step
190
Vol_down
1/1 [==============================] - 0s 19ms/step
188
Vol_down
1/1 [==============================] - 0s 19ms/step
186
Vol_down
1/1 [==============================] - 0s 20ms/step
184
Vol_up
1/1 [==============================] - 0s 20ms/step
186
Vol_up
1/1 [==============================] - 0s 22ms/step
188
Vol_down
1/1 [==============================] - 0s 20ms/step
186
Vol_down
1/1 [==============================] - 0s 23ms/step
184
Vol_down
1/1 [==============================] - 0s 21ms/step
182
Vol_down
1/1 [==============================] - 0s 21ms/step
180
Vol_down
1/1 [========================